<a href="https://colab.research.google.com/github/Chhavi0404/soc-/blob/main/aasignment3_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install kaggle
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip chest-xray-pneumonia.zip


mkdir: cannot create directory ‘/root/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  chest-xray-pneumonia.zip
replace chest_xray/__MACOSX/._chest_xray? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace chest_xray/__MACOSX/chest_xray/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# New Section

In [22]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers,models
from keras import regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [30]:
train_dir = 'chest_xray/train'
val_dir = 'chest_xray/val'
test_dir = 'chest_xray/test'

In [31]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [32]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 36, 36, 64)        0         
 ng2D)                                                           
                                                                 
 dropout_7 (Dropout)         (None, 36, 36, 64)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 34, 34, 128)      

In [33]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data = val_generator
)

Epoch 1/10
163/163 [==============================] - 96s 574ms/step - loss: 0.5795 - accuracy: 0.7423 - val_loss: 0.7004 - val_accuracy: 0.5000
Epoch 2/10
163/163 [==============================] - 70s 430ms/step - loss: 0.5362 - accuracy: 0.7515 - val_loss: 0.6879 - val_accuracy: 0.6250
Epoch 3/10
163/163 [==============================] - 69s 424ms/step - loss: 0.3788 - accuracy: 0.8232 - val_loss: 0.7647 - val_accuracy: 0.6875
Epoch 4/10
163/163 [==============================] - 69s 425ms/step - loss: 0.3263 - accuracy: 0.8554 - val_loss: 0.6698 - val_accuracy: 0.6875
Epoch 5/10
163/163 [==============================] - 69s 424ms/step - loss: 0.3043 - accuracy: 0.8643 - val_loss: 0.5963 - val_accuracy: 0.6875
Epoch 6/10
163/163 [==============================] - 68s 416ms/step - loss: 0.2943 - accuracy: 0.8689 - val_loss: 0.6719 - val_accuracy: 0.5625
Epoch 7/10
163/163 [==============================] - 68s 419ms/step - loss: 0.2636 - accuracy: 0.8823 - val_loss: 0.6636 - val_ac

In [35]:
loss,accuracy = model.evaluate(test_generator)
print("Loss:", loss)
print("Accuracy:", accuracy)

20/20 [==============================] - 4s 207ms/step - loss: 0.4110 - accuracy: 0.8205
Loss: 0.4109828770160675
Accuracy: 0.8205128312110901
